# Proof of concept of ENUM-Based Regular ISD

This code applies ENUM-Based Regular ISD on random RSD instances, measures the success probability and list sizes and compares it with theoretical estimates.
Success probabilities and list sizes are averaged over multiple RSD instances, sampled at random.
The implementation takes into account (and resolves) rounding issues.

The implementation follows the description in Appendix A.2

The code has not been optimized, we recommend using it for moderate list sizes (e.g., $\leq 2^{15}$)

In [8]:
reset(); 

load("PGE.sage"); #required to perform Partial Gaussian Elimination
load("list_ops.sage"); #required to merge lists

#Generate random RSD instance (requires that n is divided by b)
def sample_rsdp_instance(n,k,b,w):
    """
    Generate random RSD instance (requires that $n$ is divided by $b$)
    """
    F2 = GF(2);

    #Sample full rank parity-check matrix
    rank_H = 0;
    while rank_H < (n-k):
        H = random_matrix(F2,n-k,n);
        rank_H = rank(H);

    #Sample regular error vector
    e = matrix(F2,1,n);
    for i in range(w):
        pos = randrange(b);
        e[0,i*b+pos] = F2(1);

    #Compute syndrome
    s = H*e.transpose();

    return H, e, s;

##############################################

#Include the additional parity-check equations
def add_parity_checks(H, s, n, k, b, w):
    """
    Add parity-check equations encoding regularity
    Returns the modified parity-check matrix and syndrome
    """
    #Compute adapted parameters
    r = n - k;
    rp = n - k + w;
    kp = n - rp;

    ##Modify H: add partiy-check equations, remove last columns
    new_H = matrix(GF(2),rp,n);
    new_H[0:r, :] = H;
    for i in range(w):
        for j in range(b):
            new_H[r+i,i*b+j] = 1;

    new_H = new_H[:,0:n];

    ##Modify s: add new bits, corresponding to new parity-check equations
    new_s = matrix(GF(2), rp, 1);
    new_s[0:r, 0] = s;
    for i in range(w):
        new_s[r+i,0] = 1;

    return new_H, new_s;

# Theoretical estimates for ENUM-Based regular ISD
Returns the parameters to optimize the attack, as well internal quantities which will be later estimated with the simulations.

We resolve rounding issues as explained in the appendix of the paper: 

- $v = \left\lfloor\frac{k'+\ell}{w}\right\rfloor$
- $w_-$: number of blocks from which select $v$ coordinates, $w_- = w(v+1)-k'-\ell$
- $w_+$: number of blocks from which we select $v+1$ coordinates, $w_+ = w_+ = k'+\ell-wv$
- $w_-^{(1)}$: number of blocks from which we sample $v$ coordinates for the first list, $w_-^{(1)} = \lfloor w_-/2\rfloor$ 
- $w_+^{(1)}$: number of blocks from which we sample $v+1$ coordinates for the first list, $w_+^{(1)} = \lceil w_+/2\rceil$ 

Analogous notation is used for the second list:

$w_-^{(2)} = \lceil w_-/2\rceil$

$w_+^{(2)} = \lfloor w_+/2\rfloor$.

In [9]:
def enumeration_based_concrete_cost_rounding(n,k,w,b, S):
    """
    Concrete cost estimate for enumeration based ISD
    It outputs the parameters that optimize the attack, together with internal quantities:
    - parameters p and ell
    - success probability of one iteration
    - list size and number of collisions

    Input:
    - (n, k, w, b): RSD parameters, with:
    - n: code length
    - k: code dimension
    - w: Hamming weight of solution
    - b: size of weight-1 blocks
    - S: expected number of existing solutions
    """

    k_prime = k-w;  #add parity-checks

    min_cost = 1000000000000000000000000000000;
    best_params = [];


    p_max = 30;
    ell_max = 30
    
    for p in range(0,min(p_max, floor(w/2))+1,2):
        for ell in range(0, min(ell_max, n-k_prime)+1):
            
            
            v = floor((k_prime+ell)/w)
                        
            if (v+1)>b:
                continue;
            
            w_minus = w*(v+1)-k_prime-ell; w_plus = w-w_minus
            
            w1_minus = floor(w_minus/2)
            w2_minus = ceil(w_minus/2)
            
            w1_plus = ceil(w_plus/2)
            w2_plus = floor(w_plus/2)
            
            pr1 = 0; L1 = 0
            for i in range(max(0, floor(p/2)-w1_plus),min(floor(p/2),w1_minus)+1):
                term = binomial(w1_minus,i)*(1-v/b)**(w1_minus-i)*(v/b)**i*binomial(w1_plus,p/2-i)*(1-(v+1)/b)**(w1_plus-(p/2-i))*((v+1)/b)**(p/2-i)
                pr1+=term;
                L1 += binomial(w1_minus,i)*binomial(w1_plus,p/2-i)*v**i*(v+1)**(p/2-i)
                
                
            pr2 = 0; L2 = 0
            for i in range(max(0, floor(p/2)-w2_plus),min(floor(p/2),w2_minus)+1):
                term = binomial(w2_minus,i)*(1-v/b)**(w2_minus-i)*(v/b)**i*binomial(w2_plus,p/2-i)*(1-(v+1)/b)**(w2_plus-(p/2-i))*((v+1)/b)**(p/2-i)
                pr2+=term;
                L2 += binomial(w2_minus,i)*binomial(w2_plus,p/2-i)*v**i*(v+1)**(p/2-i)

            num_coll = L1*L2*2**(-ell);

            T_iter = n*(n-k_prime)**2+n*(L1+L2+num_coll);
            p_iter = pr1*pr2*S; #taking into account multiple solutions
            
            if p_iter <= 0:
                continue;
                
            cost = log(T_iter, 2)-log(p_iter, 2);
            
            #update min cost  if new cost is smaller
            if cost<min_cost:
                min_cost = N(cost);
                best_params = [p, ell, w1_minus, w1_plus, w2_minus, w2_plus, v, v+1]
                quantities = [log(L1, 2), log(L2, 2), log(num_coll, 2), log(p_iter, 2)]
                
    return min_cost, best_params, quantities

# Subroutines for ENUM-Based regular ISD

Subroutines for:
- creating lists
- creating and merging lists
- sampling a regular permutation
- checking if a vector is regular

In [10]:
def all_regular_initial_list(p_half, w_f, v_f, w_c, v_c):
    """
    Enumerate regular vectors for initial lists; the input p_half corresponds to $p/2$
    """
    vec_length = w_f*v_f+w_c*v_c; #length of the vectors we are going to enumerate
    
    all_vecs = [];
    
    #we consider i_f blocks with length v_f and weight 1, i_c = p_half - i_f blocks with length v_c and weight 1
    for i_f in range(p_half + 1):
        
        i_c = p_half - i_f;
        
        pos_v_f = cartesian_product([range(v_f) for i in range(i_f)]);
        pos_v_c = cartesian_product([range(v_c) for i in range(i_c)]);
        
        pos_unit_vectors_f = Combinations(w_f, i_f);
        pos_unit_vectors_c = Combinations(w_c, i_c);
        
        for pos_f in pos_unit_vectors_f:
            for pos_c in pos_unit_vectors_c:
                for pos_ones_f in pos_v_f:
                    for pos_ones_c in pos_v_c:
                        candidate = matrix(GF(2),1, vec_length);
                        
                        for j in range(len(pos_f)):
                            this_pos = pos_f[j]*v_f+pos_ones_f[j];
                            candidate[0, this_pos] = 1;
                        for j in range(len(pos_c)):
                            this_pos = w_f*v_f+v_c*pos_c[j]+pos_ones_c[j];
                            candidate[0, this_pos] = 1;
                            
                        all_vecs.append(candidate);
    
    return all_vecs;

######################################


def create_list_partial_sums(H_sub, list_vectors, target_s):
    """
    Create initial lists: takes as input the sub-parity check matrix and the list of vectors, 
    together with target syndrome
    """
    list_sums = [];
    for x in list_vectors:
        s = target_s + x*H_sub.transpose(); 
        list_sums.append(s);

    return list_sums;
#######################################################


def merge_lists(list_sums_left, list_sums_right, all_vecs_left, all_vecs_right):
    """
    Find collisions and return list of (merged) vectors
    """
        
    indexes = colliding_indexes(list_sums_left, list_sums_right); #indices of collisions   

    new_list_vectors = []; #new list
    for i in range(len(indexes)):
        new_vector = block_matrix(GF(2),1,2,[all_vecs_left[indexes[i][0]], all_vecs_right[indexes[i][1]]]);
        new_list_vectors.append(new_vector);
    
    return new_list_vectors;


###################################################
def ENUM_sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right):
    """
    Sample regular permutation to be used for ENUM-Based Regular ISD
    The function samples two permutations (one for the first list, one for the second list)
    Uses sample_regular_permutation as subroutine
    """
    P_list_left = sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, 0);
    P_list_right = sample_regular_permutation(n, b, v_f, v_c, w_f_right, w_c_right, b*(w_f_left + w_c_left));
    
    for i in P_list_right:
        P_list_left.append(i);
    
    
    #Now, place the remaining coordinates (i.e., those that will be moved to the last n-k' positions)
    for i in range(n):
        if i not in P_list_left:
            P_list_left.append(i);
    
    #Create permutation matrix out of P
    P = matrix(GF(2),n,n);
    for i in range(n):
        P[P_list_left[i], i] = 1;
    

    return P;

##################################################


def sample_regular_permutation(n, b, v_f, v_c, w_f, w_c, offset):
    """
    Sample regular permutation
    Sample $v_f$ coordinates from $w_f$ blocks, $v_c$ coordinates from $w_c$ blocks and move them in first positions;
    These coordinates will constitute the information set
    The input offest gives the initial block from which coordinates get sampled
    """
    P_list = []; #regular permutation
    
    #We first sample coordinates for the w_f blocks (v_f from each block)
    for i in range(w_f):
        
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_f):
            P_list.append(i*b+perm_of_b[j]+offset);
    
    #Now, sample coordinates for the w_c blocks (v_c from each block)
    for i in range(w_c):
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_c):
            P_list.append((w_f+i)*b+perm_of_b[j]+offset);
    
    return P_list;

######################

def check_regularity(x,b):
    """
    Checks if input vector is regular
    """
    n = x.ncols();
    
    is_regular = 1;
    for i in range(w):
        num_ones_in_block = x[0,i*b:(i+1)*b].list().count(1);
        if num_ones_in_block != 1:
            is_regular = 0;

    return is_regular;

# Implementation of ENUM-Based regular ISD
The implementation is only meant to be a proof of concept; it is only meant to verify the success probability, list sizes and the probabilities that the employed matrices have the desired rank 

In [11]:
def ENUM_based_ISD(n, k_prime, w, b, new_H, new_s, all_vecs_left, all_vecs_right, params, num_success, num_full_rank, num_PGE_ok, avg_num_coll):
    """
    Proof of concept ENUM-based Regular ISD; it receives as input the parity-check matrix 
    with additional equations and corresponding syndrome
    It also requires the list of vectors to prepare the initial lists (all_vecs_left, all_vecs_right)
    The input params must be formatted as [p, ell, w_f_left, w_c_left, w_f_right, w_c_right, v_f, v_c]
    """
    p = params[0]; ell = params[1]; 
    w_f_left = params[2]; w_c_left = params[3]; w_f_right = params[4]; w_c_right = params[5]; 
    v_f = params[6]; v_c = params[7];
    
    ok = 0; #ok becomes 1 when a solution is found

    #Sample regular permutation
    P = ENUM_sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right);

    #Apply permutation to H
    perm_H = new_H*P;
    
    #Do PGE, continue only if PGE does not fail
    is_full_rank, reduced_H, reduced_s = PGE(n, n-k_prime, ell, perm_H, new_s);
    
    if is_full_rank:

        num_PGE_ok += 1;

        #create first list
        list_sums_left = create_list_partial_sums(reduced_H[0:ell,0:w_f_left*v_f+w_c_left*v_c], all_vecs_left, matrix(GF(2),1,ell));
        
        #create second list
        list_sums_right = create_list_partial_sums(reduced_H[0:ell,w_f_left*v_f+w_c_left*v_c:k-w+ell], all_vecs_right, reduced_s[0:ell].transpose());
        
        #find collisions and merge vectors
        merged_vecs = merge_lists(list_sums_left, list_sums_right, all_vecs_left, all_vecs_right);
        
        avg_num_coll += len(merged_vecs); #update estimate on average number of collisions
        
        ok_found = 0;
        for x in merged_vecs:
            H_sub = reduced_H[ell:,0:k_prime+ell];
            x_right = reduced_s[ell:] - H_sub*x.transpose();
            
            w_x = x_right.list().count(1);
            if w_x == w - p: #the vector has the right weight, continue by check
                
                perm_candidate_e = block_matrix(GF(2),1,2,[x, x_right.transpose()]);
                candidate_e = perm_candidate_e*P^-1;
                
                is_regular = check_regularity(candidate_e,b);
                if is_regular:
                    ok = 1; 
    
    num_success += ok;
    
    return num_success, num_full_rank, num_PGE_ok, avg_num_coll;

# Set simulation

Select the code parameters ($n$, $k$, $b$ and $w$) and the number of RSD instances to be generated (denoted by $\mathtt{num\_instances}$)

In [12]:
#Parameters
n = 80; #code length
k = 40; #code dimension
w = 10; #number of blocks with weight 1 (must divide n)

num_instances = 1000; #number of RSD instances

b = n/w; #size of blocks (length of unit vectors forming the solution)

S = max(1,b**w/(2**(n-k))); #expected number of solutions

#Get theoretical estimates (expected list sizes and success probability, optimal values of p and s)
min_cost, params, quantities = enumeration_based_concrete_cost_rounding(n, k, w, b, S); #theoretical estimates

p = params[0]; ell = params[1]; w_f_left = params[2]; w_c_left = params[3]; w_f_right = params[4]; w_c_right = params[5]; v_f = params[6]; v_c = params[7]; 
L1 = quantities[0]; L2 = quantities[1]; num_coll = quantities[2]; p_iter = quantities[3];

print("Considering: [n, k, w, b] = "+str([n, k, w, b]));
print("Number of solutions: S = "+str(N(S)));
print("Optimal setting: p = "+str(p)+", ell = "+str(ell));
print("Th. success probability: p_iter = "+str(N(p_iter)));
print("Th. L1 = 2^"+str(N(L1))+", L2 = 2^"+str(N(L2))+", Th. Num Coll = 2^"+str(N(num_coll)));
      

Considering: [n, k, w, b] = [80, 40, 10, 8]
Number of solutions: S = 1.00000000000000
Optimal setting: p = 4, ell = 6
Th. success probability: p_iter = -3.11411300830254
Th. L1 = 2^7.01122725542325, L2 = 2^7.01122725542325, Th. Num Coll = 2^8.02245451084651


# Generating all vectors for enumeration
We generate all the vectors that we will later use for the enumeration; we do this once and then use these lists in every call to ENUM-based ISD

In [13]:
k_prime = k-w;

print("Composition of first list:");
print("-- w_f_left = "+str(w_f_left)+" blocks from which we pick v_f = "+str(v_f)+" coordinates");
print("-- w_c_left = "+str(w_c_left)+" blocks from which we pick v_c = "+str(v_c)+" coordinates");
print("-- total = "+str(w_f_left*v_f + w_c_left*v_c)+" coordinates");
print(" ");
print("Composition of second list:");
print("-- w_f_right = "+str(w_f_right)+" blocks from which we pick v_f = "+str(v_f)+" coordinates");
print("-- w_c_right = "+str(w_c_right)+" blocks from which we pick v_c = "+str(v_c)+" coordinates");
print("-- total = "+str(w_f_right*v_f + w_c_right*v_c)+" coordinates");
print(" ");
print("Total amount of coordinates = "+str(k_prime+ell));
print(" ");

all_vecs_left = all_regular_initial_list(p/2, w_f_left, v_f, w_c_left, v_c);
all_vecs_right = all_regular_initial_list(p/2, w_f_right, v_f, w_c_right, v_c);

#double check if enumeration has been done properly
num_vecs_left = sum([binomial(w_f_left,i)*v_f^i*binomial(w_c_left, p/2-i)*v_c^(p/2-i) for i in range(p/2+1)]);
num_vecs_right = sum([binomial(w_f_right,i)*v_f^i*binomial(w_c_right, p/2-i)*v_c^(p/2-i) for i in range(p/2+1)]);
print("First list: Expected size = "+str(num_vecs_left)+", Num of generated vectors = "+str(len(all_vecs_left)));
print("Second list: Expected size = "+str(num_vecs_right)+", Num of generated vectors = "+str(len(all_vecs_right)));

#Necessary later, when calling ENUM-based ISD
params = [p, ell, w_f_left, w_c_left, w_f_right, w_c_right, v_f, v_c]

Composition of first list:
-- w_f_left = 2 blocks from which we pick v_f = 3 coordinates
-- w_c_left = 3 blocks from which we pick v_c = 4 coordinates
-- total = 18 coordinates
 
Composition of second list:
-- w_f_right = 2 blocks from which we pick v_f = 3 coordinates
-- w_c_right = 3 blocks from which we pick v_c = 4 coordinates
-- total = 18 coordinates
 
Total amount of coordinates = 36
 
First list: Expected size = 129, Num of generated vectors = 129
Second list: Expected size = 129, Num of generated vectors = 129


# Start simulation and wait for results!

In [14]:
num_success = 0; #number of successful iterations
num_full_rank = 0; #num of full rank matrices in ISD
num_PGE_ok = 0; #number of H' with full ranks
avg_num_coll = 0; #average number of collisions

for id_instance in range(1, num_instances+1):
    
    #Sample RSD instance
    H, e, s = sample_rsdp_instance(n, k, b, w);

    #Add parity-chek equations
    new_H, new_s = add_parity_checks(H, s, n, k, b, w);
    
    if rank(new_H)==n-k_prime:
        num_full_rank +=1;
    

    #Launch ENUM-based ISD
    num_success, num_full_rank, num_PGE_ok, avg_num_coll = ENUM_based_ISD(n, k_prime, w, b, new_H, new_s, all_vecs_left, all_vecs_right, params, num_success, num_full_rank, num_PGE_ok, avg_num_coll);
    
    #Print comparison between theoretical and estimated
    if (id_instance%100) == 0:
        
        print("Num instances = "+str(id_instance));
        table_rows = [];
        table_rows.append(["P_iter",str(N(p_iter)),str(N(log(num_success/id_instance, 2)))]);
        table_rows.append(["Log2(Num_coll)",str(N(num_coll)),str(N(log(avg_num_coll/id_instance,2)))]);
        table_rows.append(["Cost",str(N(min_cost)),str(N(log((n-k)^2*n+n*(num_vecs_left+num_vecs_right+avg_num_coll/id_instance),2)-log(num_success/id_instance,2)))]);
        
        t = table(table_rows, header_row = [" ", " Theoretical","Empirical"])
        show(t)
        
        print("------------------------------");

Num instances = 100


,Theoretical,Empirical
P_iter,-3.11411300830254,-2.94341647163363
Log2(Num_coll),8.02245451084651,8.01730952320542
Cost,20.9954256629186,20.3132114525049


------------------------------
Num instances = 200


,Theoretical,Empirical
P_iter,-3.11411300830254,-3.39592867633114
Log2(Num_coll),8.02245451084651,8.01228931423938
Cost,20.9954256629186,20.7651102200773


------------------------------
Num instances = 300


...

------------------------------
Num instances = 900


,Theoretical,Empirical
P_iter,-3.11411300830254,-3.24392558288609
Log2(Num_coll),8.02245451084651,8.00962336972066
Cost,20.9954256629186,20.6127821264681


------------------------------
Num instances = 1000


,Theoretical,Empirical
P_iter,-3.11411300830254,-3.22431729826094
Log2(Num_coll),8.02245451084651,8.00766645369165
Cost,20.9954256629186,20.5929356134065


------------------------------
